In [1]:
import numpy as np
from data_loader import load_dataset, load_challenge, save_challenge
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from eval_script import compute_revenue

In [2]:
class BirdCrossEntropyLoss(nn.Module):
    def __init__(self, device):
        super(BirdCrossEntropyLoss, self).__init__()
        self.rev_matrix = torch.tensor(np.array(
            [[0.05, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2],
             [-0.25,  1., -0.3, -0.1, -0.1, -0.1, -0.1],
                [-0.02, -0.1,  1., -0.1, -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3,  1., -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1,  1., -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1,  1., -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1, -0.1,  1.]])).to(device)
        self.cet = nn.CrossEntropyLoss(reduction='none').to(device)


    def forward(self, input, target):
        ce_loss = self.cet(input, target)

        input_labels = torch.argmax(input, dim=1)
        target_labels = torch.argmax(target, dim=1)
        wanted = self.rev_matrix[target_labels, target_labels]
        actual = self.rev_matrix[target_labels, input_labels]
        cost = wanted - actual
        custom_loss = cost * ce_loss

        return custom_loss.mean()


In [3]:
class BirdDataset(Dataset):
    def __init__(self, birdset, is_train, device, scaler=None):
        self.device = device
        self.data = np.empty((0, 548), dtype=np.float32)
        self.labels = np.empty((0), dtype=int)
        for bird in birdset:
            if is_train:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['train_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['train_labels'])))
            else:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['test_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['test_labels'])))

        hot_matrix = np.eye(7)
        self.labels = hot_matrix[self.labels]

        if scaler == None:
            scaler = StandardScaler()
            scaler.fit(self.data)
        self.scaler = scaler

        self.data = torch.tensor(
            self.scaler.transform(self.data)).to(self.device)
        self.labels = torch.tensor(self.labels).to(self.device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y


In [4]:
def train(dataloader, model, loss_fn, optimizer, progress_steps):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if progress_steps != None and batch % progress_steps == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, show_progress):
    num_batches = len(dataloader)
    test_loss = 0
    money = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            money += compute_revenue(torch.argmax(pred, dim=1).cpu(), torch.argmax(y, dim=1).cpu())

    test_loss /= num_batches
    if show_progress:
        print(f"Avg loss: {test_loss:>8f}, Money saved: {money:.2f}$\n")
    return test_loss

def eval(model_fn, train_dataloader, test_dataloader, device, max_epochs):
    model = model_fn(None).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = BirdCrossEntropyLoss(device)
    lowest_test_loss = np.infty
    stop_criterion = 0
    for t in range(max_epochs):
        if t % 10 == 0:
            print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer, None)
        test_loss = test(test_dataloader, model, loss_fn, True)
        if test_loss > lowest_test_loss:
            stop_criterion += 1
        else:
            lowest_test_loss = test_loss
            stop_criterion = 0
        if stop_criterion >= 5:
            break
    print("Estimated performance:")
    test(test_dataloader, model, loss_fn, True)
    return model


In [5]:
model_fn = lambda _: nn.Sequential(
        nn.Linear(548, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 7)
    )

In [6]:
dataset = load_dataset()

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
train_set = BirdDataset(dataset, True, device)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, scaler)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

In [8]:
model = eval(model_fn, train_loader, test_loader, device, 500)

Epoch 1
-------------------------------
Avg loss: 0.242018, Money saved: 4144.27$

Avg loss: 0.189466, Money saved: 4590.12$

Avg loss: 0.170899, Money saved: 4726.24$

Avg loss: 0.155339, Money saved: 4914.85$

Avg loss: 0.147125, Money saved: 4976.82$

Avg loss: 0.137234, Money saved: 5129.57$

Avg loss: 0.134268, Money saved: 5136.40$

Avg loss: 0.135200, Money saved: 5077.97$

Avg loss: 0.129894, Money saved: 5163.65$

Avg loss: 0.123324, Money saved: 5266.37$

Epoch 11
-------------------------------
Avg loss: 0.120685, Money saved: 5292.11$

Avg loss: 0.118604, Money saved: 5281.97$

Avg loss: 0.116320, Money saved: 5357.31$

Avg loss: 0.114743, Money saved: 5350.15$

Avg loss: 0.113080, Money saved: 5387.23$

Avg loss: 0.112559, Money saved: 5363.80$

Avg loss: 0.112825, Money saved: 5360.27$

Avg loss: 0.108185, Money saved: 5418.79$

Avg loss: 0.108991, Money saved: 5363.57$

Avg loss: 0.106891, Money saved: 5467.03$

Epoch 21
-------------------------------
Avg loss: 0.105698

In [12]:
torch.save(model.state_dict(), 'Models/nn')

In [13]:
challenge = load_challenge()
print(challenge.shape)

(16, 3000, 548)


In [14]:
model = model_fn(None)
model.load_state_dict(torch.load('Models/nn'))
model.eval()

model = model.to(device)
results = torch.Tensor(np.empty((challenge.shape[0], challenge.shape[1]))).to(device)
for i in range(challenge.shape[0]):
    challenge_subset = challenge[i]
    challenge_subset = scaler.transform(challenge_subset)
    challenge_subset = torch.Tensor(challenge_subset).unsqueeze(0).to(device)
    preds = model(challenge_subset).squeeze()
    pred_labels = torch.argmax(preds, dim=1)
    results[i, :] = pred_labels
results = results.int().cpu().numpy()
print(results, results.shape)
save_challenge('nn', results)

[[0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 3 3]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 6 6 0]
 [0 0 0 ... 0 0 0]] (16, 3000)
